In [2]:
import json
import random
from itertools import chain
from pathlib import Path
import os
import networkx as nx
import numpy as np
import pandas as pd
import pytest

import pyciemss
from pyciemss.integration_utils.result_processing import convert_to_output_format
from pyciemss.visuals import plots, vega



/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/threadpoolctl.py:1223: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [6]:

name = "all_cluster"
timepoint_list = []
sine_list = []
sample_list = []


def save_result(data, name, ref_ext):
    """Save new reference files"""
    _output_root = 'output'
    mode = "w" if ref_ext == "svg" else "wb"
    with open(os.path.join(_output_root, f"{name}.{ref_ext}"), mode) as f:
        f.write(data)

# create sine fucntions
def return_sine_function_3(function_name):
        if function_name == "Straight":
                a, b, c, d = 0, 0, 0, np.random.randn()
                a2, b2, c2, d2 = 0, 0, 0, 0
        if function_name == "Sine_1a":
                a, b, c, d = 1*(np.random.randn() + 1), .25, np.pi, np.random.randn()
                a2, b2, c2, d2 = 1, .25, np.pi, 0
        if function_name == "Sine_2b":
                a, b, c, d = 2, .25*(np.random.randn() + 1), np.pi, np.random.randn()
                a2, b2, c2, d2 = 2, .25, np.pi, 0
        if function_name == "Sine_3c":
                a, b, c, d = 3, .25, np.pi*(np.random.randn() + 1), np.random.randn()
                a2, b2, c2, d2 = 3, .25, np.pi, 0
        def sine_function(x):
                return np.power(x, .25) + a*np.sin(b*x + c) + d
        def sine_function_default(x):
                return np.power(x, .25) + a2*np.sin(b2*x + c2) + d2
        function_parameters_individual = {'a': a,  'b': b, 'c': c,  'd': d}
        function_parameters_default = {'a': a2, 'b': b2, 'c': c2, 'd': d2}
        
        return sine_function, sine_function_default, function_parameters_individual, function_parameters_default

function_dataframe = pd.DataFrame(columns=['functions', 'sample_id', 'a', 'b',  'c',  'd'])
function_list = []
j = 0
all_functions ={}

# create sine dataframes and save the parameters used
for funct_name in ["Straight", "Sine_1a", "Sine_2b", "Sine_3c"]:
        for _ in range(100):
                j+=1
                timepoint_list.extend([x for x in range(100)])
                sample_list.extend([j]*100)
                random_function,  sine_function_default, function_parameters_individual, function_parameters_default = return_sine_function_3(funct_name)

                function_parameters_individual['functions'] = funct_name
                function_parameters_individual['sample_id'] = j
                function_list.append(function_parameters_individual)
                sine_list.extend([random_function(x) for x in range(100)])
        all_functions[funct_name] = sine_function_default 
        function_parameters_default['functions'] = funct_name + " Default"
        function_parameters_default['sample_id'] = None

#keep track of parameters
function_dataframe = pd.DataFrame(function_list)
function_default_dataframe =  pd.DataFrame(function_parameters_default, index=[0])


sine_distribution= pd.DataFrame({'timepoint_id' : timepoint_list,
                   'sample_id' : sample_list,
                   'Example': sine_list})


traces_list = []   

# get examplary lines
#change kmean here to use clustering
examplary_line, trajectory_dict = pyciemss.visuals.trajectories.select_traces(sine_distribution, kmean=True)
# get the sample'id for the examplary data used per sample_id and select_by
best_samples_cluster = examplary_line[['sample_id', 'select_by', 'cluster']].drop_duplicates()
best_samples_df_list = []

# for each cluster create a new plot
for cluster in list(np.unique(examplary_line['cluster'])): 
        # for distribution only use the sample_ids in the cluster
       
        cluster_sample_id = trajectory_dict[cluster.lower()]['cluster_sample_id']  
        cluster_sine_distribution =  sine_distribution[sine_distribution["sample_id"].isin(cluster_sample_id)]
        current_cluster_df = examplary_line[examplary_line['cluster'] == cluster]
        current_cluster_df = current_cluster_df.reset_index()
        # get right format to use as traces
        examplary_lines_pivot = current_cluster_df[['examplary_line', 'mean_trajectory', 'timepoint', 'select_by']].pivot_table(
        values="examplary_line", index = ["timepoint", 'mean_trajectory'], columns="select_by"
        )
        examplary_lines_pivot = examplary_lines_pivot.reset_index()
        examplary_lines_pivot = examplary_lines_pivot.rename(columns = {'mean_trajectory': 'Baseline_Trajectory_Mean'})

        # # get baseline for sine functions
        # for function_name, function in all_functions.items():
        #         examplary_lines_pivot["Baseline_" + function_name.title()] = [function(x) for x in range(100)]
        # plot the traces along with the baselines
        schema = plots.trajectories(cluster_sine_distribution[['timepoint_id', 'sample_id', 'Example']], traces = examplary_lines_pivot)
        path = "{}_{}".format(name, cluster)
        image = plots.ipy_display(schema, format="PNG").data
        save_result(image, path, "png")
        plots.ipy_display(schema, format="interactive")

# clean up table to get the parameters of the samples id's used for examplary
all_clusters = best_samples_cluster.merge(function_dataframe, how="inner", on="sample_id")
all_clusters = pd.concat([all_clusters, function_default_dataframe])[['functions', 'select_by', 'a', 'b', 'c', 'd']]
all_clusters = all_clusters.rename(columns = {'functions': 'Function Name', 'select_by': "Selection Method"})
all_clusters["Function Name"] = all_clusters["Function Name"].apply(lambda x: x.replace("_", " ").title())
all_clusters["Examplar #"] = all_clusters["Selection Method"].apply(lambda x: 1 if len(str(x).split("_")) < 2 else str(x).split("_")[1])
all_clusters["Selection Method"] = all_clusters["Selection Method"].apply(lambda x:  str(x).split("_")[0])
all_clusters = all_clusters.round(2)


all_clusters.to_csv("output/{}.csv".format(name))
# table with sine variables of starting function and the formula of the best select_by lines and metrics1

Init 1


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Do

26.552 --> 

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 1249 tasks      | elapsed:    0.3s


13.393 --> 

...

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

ssr based F test:         F=-7.2000 , p=1.0000  , df_denom=72, df_num=10
ssr based chi2 test:   chi2=-90.0000, p=1.0000  , df=10
likelihood ratio test: chi2=-1762.2977, p=1.0000  , df=10
parameter F test:         F=0.0002  , p=1.0000  , df_denom=72, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.2000 , p=1.0000  , df_denom=72, df_num=10
ssr based chi2 test:   chi2=-90.0000, p=1.0000  , df=10
likelihood ratio test: chi2=-1917.0139, p=1.0000  , df=10
parameter F test:         F=0.0000  , p=1.0000  , df_denom=72, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.2000 , p=1.0000  , df_denom=72, df_num=10
ssr based chi2 test:   chi2=-90.0000, p=1.0000  , df=10
likelihood ratio test: chi2=-2497.2790, p=1.0000  , df=10
parameter F test:         F=0.0000  , p=1.0000  , df_denom=72, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.2000 , p=1.0000  , df_denom=72, df_num=10
ssr based

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

ssr based F test:         F=-7.6930 , p=1.0000  , df_denom=77, df_num=10
ssr based chi2 test:   chi2=-89.9177, p=1.0000  , df=10
likelihood ratio test: chi2=-629.7646, p=1.0000  , df=10
parameter F test:         F=20578.1199, p=0.0000  , df_denom=77, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.6932 , p=1.0000  , df_denom=77, df_num=10
ssr based chi2 test:   chi2=-89.9210, p=1.0000  , df=10
likelihood ratio test: chi2=-633.3850, p=1.0000  , df=10
parameter F test:         F=26045.9614, p=0.0000  , df_denom=77, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.6866 , p=1.0000  , df_denom=77, df_num=10
ssr based chi2 test:   chi2=-89.8439, p=1.0000  , df=10
likelihood ratio test: chi2=-572.1226, p=1.0000  , df=10
parameter F test:         F=44406.8559, p=0.0000  , df_denom=77, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.6870 , p=1.0000  , df_denom=77, df_num=10
ssr ba

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '


In [1]:
def distributions():
    model_1_path = (
            "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration"
            "/main/data/models/SEIRHD_NPI_Type1_petrinet.json"
    )
    start_time = 0.0
    end_time = 100.0
    logging_step_size = 1
    num_samples = 30
    sample = pyciemss.sample(
        model_1_path,
        end_time,
        logging_step_size,
        num_samples,
        start_time=start_time,
        solver_method="euler",
    )["unprocessed_result"]

    for e in sample.values():
        if len(e.shape) > 1:
            num_timepoints = e.shape[1]

    return convert_to_output_format(
        sample,
        timepoints=np.linspace(start_time, end_time, num_timepoints),
        time_unit="notional",
    )

distributions = distributions()
print(distributions.head())
print(distributions.columns)
all_columns = ["timepoint_id", "sample_id", 'infected_observable_state',
       'exposed_observable_state', 'hospitalized_observable_state',
       'dead_observable_state']
all_columns = ["timepoint_id", "sample_id", 'infected_observable_state']

distributions = distributions[all_columns]
distributions

NameError: name 'pyciemss' is not defined

In [13]:
#'timepoint_id', 'sample_id'
sine_distribution = pd.read_csv("paper_sine_05/paper_sine_05_new_df.csv")
sine_distribution = sine_distribution.rename(
            columns={
                'Unnamed: 0.1': "timepoint_id",
                'Unnamed: 0': "100",
            }
        )
sine_distribution = sine_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")

traces_list = []   
 
#select_by "mean", "var", "granger"
for select_by in ['mean', 'var', 'granger']:
    examplary_line, mean_line_df = pyciemss.visuals.trajectories.select_traces(sine_distribution, select_by = select_by,  kmean=True)
    traces_list.append(examplary_line)


new_df = pd.concat(traces_list, axis = 1)
combined_df = pd.concat([new_df, mean_line_df], axis =1)

schema = plots.trajectories(sine_distribution, traces = combined_df,  keep=".*_state")

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")

FileNotFoundError: [Errno 2] No such file or directory: 'paper_sine_05/paper_sine_05_new_df.csv'